In [1]:
from autodiff.structures import Number
from autodiff.structures import Array
from autodiff.optimizations import bfgs_symbolic
from autodiff.optimizations import bfgs
from autodiff.optimizations import steepest_descent
import timeit
import numpy as np

We implemented three optimization methods in our optimization module, steepest descent, AD BFGS and symbolic BFGS. The former two use automatic differentiation to obtain gradient information and the last one requires user input of the expression of the gradient.

In this example, we use the classic rosenbrock function to benchmark each optimization method. We time each method to compare efficiency.

Let's first take a look at the AD bfgs method: Use Array([Number(2),Number(1)]) as initial guess


In [2]:

initial_guess = Array([Number(2),Number(1)])

def rosenbrock(x0):
    return (1-x0[0])**2+100*(x0[1]-x0[0]**2)**2

initial_time2 = timeit.timeit()

results = bfgs(rosenbrock,initial_guess)
print("Xstar:",results[0])
print("Minimum:",results[1])
print("Jacobian at each step:",results[2])
final_time2 = timeit.timeit()

time_for_optimization = initial_time2-final_time2
print('\n\n\n')
print("Time for symbolic bfgs to perform optimization",time_for_optimization,'total time taken is',time_for_optimization)
      
      
      
      

Xstar: [Number(val=1.0000000000025382) Number(val=1.0000000000050797)]
Minimum: Number(val=6.4435273497518935e-24)
Jacobian at each step: [array([2402, -600]), array([-5.52902304e+12, -1.15187980e+09]), array([-474645.77945484,  127109.93018289]), array([ 1.62663315e+09, -2.70845802e+07]), array([-8619.39185109,  2161.73842208]), array([-144.79886656,   36.76686628]), array([1.99114433e+00, 3.55840767e-04]), array([1.99094760e+00, 4.05114252e-04]), array([1.96305014, 0.00739234]), array([1.9349555 , 0.01442883]), array([1.87896168, 0.02845251]), array([1.79486979, 0.04951253]), array([1.65477644, 0.08459553]), array([1.43057943, 0.14073564]), array([1.06628965, 0.23194665]), array([0.47793023, 0.37924961]), array([-0.47390953,  0.61758141]), array([-2.01036637,  1.00259974]), array([-4.48450951,  1.62438092]), array([-8.45367483,  2.63103416]), array([-14.76319038,   4.28099834]), array([-4.62373441,  1.75866303]), array([141.6035648 , -30.46772268]), array([ 7.1036742 , -1.70213995]),

The total time needed for the entire optimization process is around 0.0003 s.

Then, let's compare with the traditional symbolic optimization using bfgs

First, the user needs to calculate the derivative either by hand or through sympy. Here we use sympy.

In [8]:
from sympy import * 
import sympy

initial_time = timeit.timeit()

x, y = symbols('x y') 

rb = (1-x)**2+100*(y-x**2)**2
print("Function to be derivatized : {}".format(rb)) 

# Use sympy.diff() method 
par1 = diff(rb, x) 
par2 = diff(rb,y)
print("After Differentiation : {}".format(par1)) 
print("After Differentiation : {}".format(par2)) 

def gradientRosenbrock(x0):
    x=x0[0]
    y=x0[1]
    drdx = -2*(1 - x) - 400*x*(-x**2 + y)
    drdy = 200 *(-x**2 + y)
    return drdx,drdy

final_time=timeit.timeit()
time_for_sympy = initial_time-final_time
print("Time for sympy to find derivative expression",time_for_sympy)

Function to be derivatized : (1 - x)**2 + 100*(-x**2 + y)**2
After Differentiation : -400*x*(-x**2 + y) + 2*x - 2
After Differentiation : -200*x**2 + 200*y
Time for sympy to find derivative expression 0.0014529649999914795


The time taken for sympy to find the derivative is around 0.0003 second and this can be a lot more if the user calculates derivatives by hand.

Second, use symbolic bfgs to perform optimization. Use [2,1] as the initial guess.


In [9]:
initial_time1 = timeit.timeit()

results = bfgs_symbolic(rosenbrock,gradientRosenbrock,[2,1])
print("Xstar:",results[0])
print("Minimum:",results[1])
print("Jacobian at each step:",results[2])
final_time1 = timeit.timeit()

time_for_optimization_symbolic = initial_time1-final_time1
print('\n\n\n')
print("Time for symbolic bfgs to perform optimization",time_for_optimization_symbolic,'total time taken is',time_for_optimization_symbolic+time_for_sympy)







Xstar: [1. 1.]
Minimum: 6.4435273497518935e-24
Jacobian at each step: [array([2402, -600]), array([-5.52902304e+12, -1.15187980e+09]), array([-474645.77945484,  127109.93018289]), array([ 1.62663315e+09, -2.70845802e+07]), array([-8619.39185109,  2161.73842208]), array([-144.79886656,   36.76686628]), array([1.99114433e+00, 3.55840767e-04]), array([1.99094760e+00, 4.05114252e-04]), array([1.96305014, 0.00739234]), array([1.9349555 , 0.01442883]), array([1.87896168, 0.02845251]), array([1.79486979, 0.04951253]), array([1.65477644, 0.08459553]), array([1.43057943, 0.14073564]), array([1.06628965, 0.23194665]), array([0.47793023, 0.37924961]), array([-0.47390953,  0.61758141]), array([-2.01036637,  1.00259974]), array([-4.48450951,  1.62438092]), array([-8.45367483,  2.63103416]), array([-14.76319038,   4.28099834]), array([-4.62373441,  1.75866303]), array([141.6035648 , -30.46772268]), array([ 7.1036742 , -1.70213995]), array([10.42215839, -2.72209527]), array([13.5437883 , -3.72650641]

The total time taken for symbolic optimization is 0.007416142000003845 s, while the total time taken for a0.0005920969999999581